In [20]:
# imports
import os
import json
import re

import torch 
from torch.utils import data
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import numpy as np

from MDP import MDP

import stable_baselines3
import sb3_contrib

import gym

In [21]:
# check torch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3080'

In [22]:
# create Neural Network

class Net(nn.Module):
    """
    input : 2 X 4 X 4 grid
    label : Move [0;6]
    """
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(32, 64)

        self.fc2 = nn.Linear(64, 128)

        self.out = nn.Linear(128, 32)


    def forward(self, x):
        x = x.float()

        x = F.relu(self.fc1(x))

        x = F.relu(self.fc2(x))

        x = self.out(x)
    
        return x      

In [23]:
#creating model
net = Net()
net.cuda()
print(net)

params = list(net.parameters())
print(f"number of parameters: {len(params)}")

#loss function
loss = nn.CrossEntropyLoss()

#optimizer
optimizer = torch.optim.Adam(net.parameters())
optimizer

Net(
  (fc1): Linear(in_features=32, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=128, bias=True)
  (out): Linear(in_features=128, out_features=32, bias=True)
)
number of parameters: 6


Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

In [24]:
#custom environment
from gym import spaces

class Gridworld(gym.Env):

    metadata = {"render.modes" : ["human"]}

    def __init__(self, episodes, dir = ["data_easy", "generated_easy", "data_medium", "generated_med", "data", "generated_imitation"], type = "train", 
                p = [0.15, 0.15, 0.175, 0.175, 0.25, 0.2], lambda1 = 0.01, lambda2 = 0.1, lambda3 = 1, load_optimal = False, epsilon = 0.1) -> None:
        super(Gridworld, self).__init__()
        self.action_space = spaces.Discrete(6)
        self.observation_space = spaces.Box(low = 0, high = 10, shape = (11, 4, 4))

        #available MDPs
        self.dir = dir
        self.type = type
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.lambda3 = lambda3

        self.actions = ["move", "turnLeft", "turnRight", "pickMarker", "putMarker","finish"]
        self.actionsDict = {"move" : 0, "turnLeft" : 1, "turnRight" : 2, "pickMarker" : 3, "putMarker" : 4, "finish" : 5}

        self.epsilon = epsilon

        self.load_optimal = load_optimal

        #generate curriculum
        #{"data_easy" : 15% ,"generated_easy" : 20% , "data_medium" : 15% , "generated_med" : 15% "data" : 30%, "generated_imitation" : 15%}
        self.curriculum = []
        #data_easy
        for directory,amount in zip(dir, p):
            tasks = list(np.random.choice(os.listdir(os.sep.join(["datasets", directory, type, "task"])), int(episodes * amount)))
            tasks = zip([directory for _ in range(int(episodes * amount))], tasks)
            self.curriculum += tasks
        self.currentTask = 0
    

    def construct_feature_matrices(self, matrix):
        """
        creates the input for the Neural Network: 11 x 4 x 4 boolean tensor for:
        - if a wall is present
        - if a marker in the pregrid is present
        - if a marker in the postgrid is present
        - if the agent facing left, down, right, up in the pregrid (one for each direction)
        - if the agent facing left, down, right, up in the postgrid (one for each direction)
        """
        wallMat = matrix[0] == 10

        marker_pre = (matrix[0] > 4) & (matrix[0] < 10)
 
        marker_post = (matrix[1] > 4) & (matrix[1] < 10)

        agent_pre = np.zeros((4,4,4))
        agent_dir = (matrix[(0, *self.currentMDP.agentPosition)] - 1) % 4
        agent_pre[(int(agent_dir), *self.currentMDP.agentPosition)] = 1
        
        agent_post = np.zeros((4,4,4))
        agent_pos_post = ((matrix[1] > 0) & (matrix[1] < 9)).nonzero()
        agent_dir = (matrix[(1, *agent_pos_post)] - 1) % 4
        agent_post[(int(agent_dir), *agent_pos_post)] = 1
        
        return np.array([wallMat, marker_pre, marker_post, *agent_pre, *agent_post])


    def reset(self):
        nextDir, self.nexti = self.curriculum[self.currentTask]
        self.nexti = re.sub(r"\D", "", self.nexti)
        self.currentTask += 1
        self.currentMDP = MDP(nextDir, self.type, self.nexti, lambda1= self.lambda1, lambda2 = self.lambda2, lambda3 =self.lambda3)

        # load optimal sequence if possible
        try:
            with open(os.sep.join(["datasets", nextDir, self.type, "seq", self.nexti + "_seq.json"])) as grid:
                grid = json.load(grid)
                self.optimal_seq = grid["sequence"]
                # current step
                assert self.optimal_seq[-1] == "finish"
        except:  
            self.optimal_seq = []
        self.steps = 0

        #with probability 1 - epsilon use either the optimal sequence or better masks
        self.use_optimal =  (np.random.rand() < (1 - self.epsilon)) and self.load_optimal

        return self.construct_feature_matrices(self.currentMDP.get_current_state())

    def step(self, action):
        nextState, rew, done, info = self.currentMDP.sample_next_state_and_reward(self.actions[action])
        self.steps += 1
        if self.steps > 500:
            return nextState, -self.lambda3, True, info 

        return self.construct_feature_matrices(nextState), rew - 0.01, done, info 
        
    def render(self):
        self.currentMDP.print_grid()

    def close(self):
        pass
    
    def action_masks(self):

        mat = self.currentMDP.get_current_state()
        if np.array_equal(mat[0], mat[1]):
            return np.array([0,0,0,0,0,1])

        # force agent to take optimal action if possible
        mask = self.currentMDP.action_mask()

        if self.use_optimal and self.optimal_seq:
            mask = np.zeros(6)
            mask[self.actionsDict[self.optimal_seq[self.steps]]] = 1
            return mask
        
        return mask

    # functions bellow are only used for inheritance 
    def get_MDP(self):
        return self.currentMDP

    def get_MDP_name(self):
        return self.nextDir, self.nextType, self.nexti

In [25]:
testEnv = Gridworld(100, dir = ["generated_imitation"], load_optimal= True)

In [26]:
m = testEnv.reset()
testEnv.render()
print(testEnv.action_masks())
m, r, _, _ = testEnv.step(np.argmax(testEnv.action_masks()))
print(r)
testEnv.render()
testEnv.action_masks()
_, r, _, _ = testEnv.step(np.argmax(testEnv.action_masks()))
print(r)
testEnv.render()

[[['.' '.' '.' '#']
  ['.' '.' '<' '.']
  ['O' '#' '.' '.']
  ['.' '.' '#' '.']]

 [['.' '.' '.' '#']
  ['.' '.' '.' '.']
  ['O' '#' '.' '.']
  ['<' '.' '#' '.']]]
[1. 0. 0. 0. 0. 0.]
0.0
[[['.' '.' '.' '#']
  ['.' '<' '.' '.']
  ['O' '#' '.' '.']
  ['.' '.' '#' '.']]

 [['.' '.' '.' '#']
  ['.' '.' '.' '.']
  ['O' '#' '.' '.']
  ['<' '.' '#' '.']]]
0.0
[[['.' '.' '.' '#']
  ['<' '.' '.' '.']
  ['O' '#' '.' '.']
  ['.' '.' '#' '.']]

 [['.' '.' '.' '#']
  ['.' '.' '.' '.']
  ['O' '#' '.' '.']
  ['<' '.' '#' '.']]]


In [56]:
np.random.seed(123)

def test_RL_models(model):
    totalCorrect, totalOptimal = 0,0
    for dir, num in zip(["data_easy", "data_medium", "data"], [80, 24, 480]):
        print("current data: " + dir)
        total_steps = 0
        valDataset = Gridworld(num * 10, dir = [dir], p = [1], type = "val", lambda1=0, lambda2=0, load_optimal= False)
        correct, total, optimal = 0, num*10, 0
        for task in range(int(total)):
            if task % num == num-1:
                print(f"{(task+1) / total *100} %, running acc: {(correct*100)/(task+1)}, task solved optimaly: {optimal*100/task} %, average steps to solve: {total_steps / task}")
            currMDP = valDataset.reset()
            lenOptimalSeq = len(valDataset.optimal_seq)
            done = False
            steps = 0
            while not done and steps < 50:
                action = model.predict(currMDP, action_masks = valDataset.action_masks())[0]
                currMDP, rew, done, _ = valDataset.step(action)
                steps += 1
                if rew > 0:
                    correct += 1
                    total_steps += steps
                    if steps == lenOptimalSeq:
                        optimal += 1
        totalCorrect += correct
        totalOptimal += optimal
    print(f"Total Accuracy : {totalCorrect*100/5400}, Solved Optimally : {totalOptimal *100 / 5400}")

In [28]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from sb3_contrib.ppo_mask import MaskablePPO
from stable_baselines3.common.env_util import make_vec_env

import torch.nn.functional as F
from torch import nn

class CustomFeatureExtractorTorch(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.Space, features_dim: int = 32):
        super().__init__(observation_space, features_dim)
        
        self.fc1 = net.fc1
        self.fc2 = net.fc2
        self.out = net.out
  
    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.out(x))

        return x

net_arch = [
    32, 64,
    dict(vf = [64, 32, 18], pi = [32, 16, 6])
]

policy_kwargs = dict(
    features_extractor_class = CustomFeatureExtractorTorch,
    net_arch = net_arch
)

In [29]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from sb3_contrib.ppo_mask import MaskablePPO
from stable_baselines3.common.env_util import make_vec_env
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy

from stable_baselines3.ppo import PPO
from stable_baselines3.common.policies import ActorCriticPolicy

In [30]:
policy_kwargs = dict(
    features_extractor_class = CustomFeatureExtractorTorch,
    net_arch = net_arch
)

In [61]:
episodes = 1e6

FinalEnv = make_vec_env(Gridworld, n_envs= 2  , 
    env_kwargs={"episodes" : episodes, "lambda1" : 0.1, "lambda2" : 0.25, "lambda3": 1, 
            "dir" : ["data_easy", "data_medium", "data", "generated_imitation", "generated_easy", "generated_med"], "load_optimal" : True, "epsilon" : 0.5})

FinalModel = MaskablePPO(MaskableActorCriticPolicy, FinalEnv,  verbose = 1, n_steps= 500)#policy_kwargs = policy_kwargs,

FinalModel.learn(episodes, log_interval = 10000)

Using cuda device


In [248]:
FinalModel.learn(1.5 * 1e5)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 13.9     |
|    ep_rew_mean     | 1.14     |
| time/              |          |
|    fps             | 137      |
|    iterations      | 1        |
|    time_elapsed    | 14       |
|    total_timesteps | 2000     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 23.5        |
|    ep_rew_mean          | 1.02        |
| time/                   |             |
|    fps                  | 174         |
|    iterations           | 2           |
|    time_elapsed         | 22          |
|    total_timesteps      | 4000        |
| train/                  |             |
|    approx_kl            | 0.038805638 |
|    clip_fraction        | 0.176       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.415      |
|    explained_variance   | 0.504       |
|    learning_rate        | 0.

In [57]:
test_RL_models(FinalModel)

current data: data_easy
10.0 %, running acc: 91.25, task solved optimaly: 64.55696202531645 %, average steps to solve: 5.405063291139241
20.0 %, running acc: 91.25, task solved optimaly: 62.264150943396224 %, average steps to solve: 5.276729559748428
30.0 %, running acc: 90.41666666666667, task solved optimaly: 62.76150627615063 %, average steps to solve: 5.121338912133891
40.0 %, running acc: 90.0, task solved optimaly: 62.69592476489028 %, average steps to solve: 4.858934169278997
50.0 %, running acc: 90.0, task solved optimaly: 62.40601503759399 %, average steps to solve: 5.110275689223058
60.0 %, running acc: 89.79166666666667, task solved optimaly: 63.25678496868476 %, average steps to solve: 5.039665970772442
70.0 %, running acc: 90.35714285714286, task solved optimaly: 63.864042933810374 %, average steps to solve: 5.15742397137746
80.0 %, running acc: 91.09375, task solved optimaly: 65.41471048513301 %, average steps to solve: 5.098591549295775
90.0 %, running acc: 90.6944444444

In [37]:
testEnv = Gridworld(100,load_optimal= False)

In [55]:
curr = testEnv.reset()
testEnv.render()

[[['.' '.' '.' '#']
  ['.' '#' '.' '#']
  ['#' '.' '>' '.']
  ['#' '.' '.' '#']]

 [['.' '.' '.' '#']
  ['.' '#' '.' '#']
  ['#' '.' '.' '^']
  ['#' '.' '.' '#']]]


In [54]:
a = FinalModel.predict(curr, action_masks= testEnv.action_masks(), deterministic= False)[0]
curr, _, done, _ = testEnv.step(a)
print(testEnv.action_masks())
testEnv.render()
if done:
    print("finished")

[1. 1. 1. 0. 0. 0.]
[[['.' '#' 'v' '.']
  ['.' '.' '.' '.']
  ['.' '.' '.' '#']
  ['.' '.' '#' '.']]

 [['.' '#' '.' '.']
  ['.' '^' '.' '.']
  ['.' '.' '.' '#']
  ['.' '.' '#' '.']]]
